# TER
## Système de recommandation de livres basé sur le contenu

# Cahier des charges

Projet : Système de recommandation de livres

Objectif : 
- Prédire les livres qu'un utilisateur pourrait aimer en fonction de ses préférences (recommandation     collaborative) et des similarités avec d'autres livres (recommandation basée sur le contenu)

Techniques Utilisées :
- Filtrage collaboratif (basé-mémoire/basé-modèle)
- Recommandation basée sur le contenu (TF-IDF, embeddings : Word2Vec, GloVe, etc)
- Modèles : KNN, SVD, NMF, etc

Étapes :

Dataset
- Utiliser : https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset?select=Users.csv
- Tester d’autres datasets de recommandation de livres

Prétraitement
- Nettoyage des données (valeurs manquantes, doublons)
- Feature engineering (genres, auteurs, notes moyennes)
- Vectorisation des textes (titres, descriptions) avec TF-IDF, Word2Vec, GloVe, etc

Algorithmes
- Collaboratif : matrice utilisateur-livre, SVD, NMF, KNN, etc
- Basé sur le contenu : similarité cosinus sur les features textuelles

Évaluation 
- RMSE, précision des notes prédites, etc

Interface Graphique
- Exploration des données 
    - Chargement de données
    - Affichages des statistiques sur les données (analytique et graphique)

- Entrainement 
    - Choix de la méthode (basée contenu, collaborative), choix du modèle, paramétrage
    - Affichage des métriques

- Test 
    - Créer un profil avec un ID utilisateur et ses notations, afficher les recommandations collaboratives et celles basées sur le contenu
    - Saisie d’un ID utilisateur existant et prédire des recommandations collaboratives et celles basées sur le contenu

## Problématique

Comment concevoir un système de recommandation de livres efficace, capable d'exploiter le contenu textuel pour proposer des ouvrages similaires à ceux appréciés par un lecteur, sans dépendre de l'historique global des utilisateurs ?

## Objectif

Prédire les livres qu'un utilisateur pourrait aimer en fonction de ses préférences (recommandation collaborative) et des similarités avec d'autres livres (recommandation basée sur le contenu)

## Exploration des données

## Prétraitement

- Nettoyer les données (dropna, enlever les stop words (nltk ou sk-learn)...)
- Eviter la casse
- Tokenisation

## Partie basée sur le contenu

### Modèle

- Utilisation de Word2Vec (gensim models)
(pas sûr mais peut-être passer à BERT pour du deep learning) (TF-IDF)
- Entrainement du modèle sur les données

### Système de recommandation

- Calculer la similarité (cosine silimarity ou retourner les vecteurs les plus proches)

## Partie recommandation collaborative

### Modèle

- SVD (Surprise), Matrix Factorization-based algorithms

### Système de recommandation

- Prendre n premiers éléments renvoyés dans la prédiction du modèle

## Partie recommandation hybride

- Combiner les deux scores pour recommander un livre (addition avec un poids alpha qui pourrait servir à contrôler l'influence du type de recommandation)

- Renvoie les n meilleures recommandations

## Interface graphique

- Streamlit/Flask

# A reflechir

### Apprentissage supervisé

- classes basées sur le genre -> filtrer lors de la recommandation en focntion de celui-ci

# Comparaison des modèles

Comparer les modèles (une 10aine) pour la partie collaborative, optimiser les hyper paramètres etc

# Application

Ajout de la création d'un profil

In [1]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD

In [2]:
path = "./datasets/"

books = pd.read_csv(path+"Books.csv")
users= pd.read_csv(path+"Users.csv")
ratings = pd.read_csv(path+"Ratings.csv")

C:\Users\solut\AppData\Local\Temp\ipykernel_14184\1818534166.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(path+"Books.csv")


In [3]:
def show_image(url, width=100):
    return f'<img src="{url}" width="{width}">'

def url_to_img(df, func):
    return df.style.format({'Image-URL-S':func, 'Image-URL-M':func, 'Image-URL-L':func}, escape=False)


In [4]:
books.head().style.format({'Image-URL-S':show_image, 'Image-URL-M':show_image, 'Image-URL-L':show_image})

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,,,
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,,,
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,,,
3,0374157065,Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It,Gina Bari Kolata,1999,Farrar Straus Giroux,,,
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,,,


In [5]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [6]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [7]:
new_books=books
new_books["Book-Title"]=new_books["Book-Title"].str.lower()
new_books["Book-Author"]=new_books["Book-Author"].str.lower()
new_books["Publisher"]=new_books["Publisher"].str.lower()
new_books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,classical mythology,mark p. o. morford,2002,oxford university press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,clara callan,richard bruce wright,2001,harperflamingo canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,decision in normandy,carlo d'este,1991,harperperennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,flu: the story of the great influenza pandemic...,gina bari kolata,1999,farrar straus giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,the mummies of urumchi,e. j. w. barber,1999,w. w. norton &amp; company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,there's a bat in bunk five,paula danziger,1988,random house childrens pub (mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,from one to one hundred,teri sloat,1991,dutton books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,lily dale : the true story of the town that ta...,christine wicker,2004,harpersanfrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,republic (world's classics),plato,1996,oxford university press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [8]:
reader = Reader(rating_scale=(0, 10))
data=Dataset.load_from_df(ratings[["User-ID", "ISBN", "Book-Rating"]], reader)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

books_reco=books[["ISBN", "Book-Title", "Book-Author", "Year-Of-Publication", "Publisher"]]
tfidf=TfidfVectorizer()
matrice = tfidf.fit_transform(books_reco["Book-Title"])

books_reco

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,classical mythology,mark p. o. morford,2002,oxford university press
1,0002005018,clara callan,richard bruce wright,2001,harperflamingo canada
2,0060973129,decision in normandy,carlo d'este,1991,harperperennial
3,0374157065,flu: the story of the great influenza pandemic...,gina bari kolata,1999,farrar straus giroux
4,0393045218,the mummies of urumchi,e. j. w. barber,1999,w. w. norton &amp; company
...,...,...,...,...,...
271355,0440400988,there's a bat in bunk five,paula danziger,1988,random house childrens pub (mm)
271356,0525447644,from one to one hundred,teri sloat,1991,dutton books
271357,006008667X,lily dale : the true story of the town that ta...,christine wicker,2004,harpersanfrancisco
271358,0192126040,republic (world's classics),plato,1996,oxford university press


In [10]:
matrice.shape

(271360, 78765)

In [ ]:
query="the"

def search(query,vectorizer):
    processed = re.sub("[^a-zA-Z0-9 ]", "", query.lower())
    query_vec = vectorizer.transform([query])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -10)[-10:]
    results = titles.iloc[indices]
    results = results.sort_values("ratings", ascending=False)

    return results.head(5).style.format({'url': make_clickable, 'cover_image': show_image})

In [10]:
#Recommandation basée sur le contenu

vect = TfidfVectorizer()
#matrice = vect.fit_transform(books_reco["Content"])

In [14]:
from sklearn.neighbors import NearestNeighbors

# Initialisation avec la similarité cosinus
nn = NearestNeighbors(n_neighbors=5, metric='cosine', algorithm='brute')
nn.fit(matrice)  # Pas de .toarray() ici ! Sparse OK

# Exemple : chercher les 5 documents les plus proches du document 42
distances, indices = nn.kneighbors(matrice[10], n_neighbors=10)

# Afficher résultats
for i, (idx, dist) in enumerate(zip(indices[0], distances[0])):
    print(f"{i+1}. Document {idx} - Similarité = {1 - dist:.4f}")


1. Document 10 - Similarité = 1.0000
2. Document 32387 - Similarité = 0.5091
3. Document 43359 - Similarité = 0.5091
4. Document 60227 - Similarité = 0.4760
5. Document 37379 - Similarité = 0.4760
6. Document 4300 - Similarité = 0.4672
7. Document 92800 - Similarité = 0.4672
8. Document 125201 - Similarité = 0.4460
9. Document 54792 - Similarité = 0.4456
10. Document 141337 - Similarité = 0.4445


In [ ]:
#similarity = cosine_similarity(matrice)
#similarity

In [ ]:
#recommandation collaborative

svd=SVD()
train_set=data.build_full_trainset()
svd.fit(train_set)

In [ ]:
# Fonction de recommandation
def reco_collab(user_id, n):
    # Construire les éléments non encore notés par l'utilisateur
    anti_testset = train_set.build_anti_testset()

    # Filtrer pour ne garder que les éléments pour cet utilisateur
    user_testset = [entry for entry in anti_testset if entry[0] == user_id]
    print(user_testset)

    # Prédire les notes
    predictions = svd.test(user_testset)

    # Trier par estimation décroissante
    predictions.sort(key=lambda x: x.est, reverse=True)

    # Récupérer les n meilleurs items
    reco = [pred.iid for pred in predictions[:n]]
    return reco

In [ ]:
def reco_collab(user_id, n):
    user_items = set(j for (j, _) in train_set.ur[user_id])

    anti_testset_user = [
        (user_id, train_set.to_raw_iid(i), 0)  # 0 est une note fictive
        for i in train_set.all_items()
        if i not in user_items
    ]

    predictions = svd.test(anti_testset_user)

    predictions.sort(key=lambda x: x.est, reverse=True)

    reco = [pred.iid for pred in predictions[:n]]
    return reco


In [ ]:
reco_collab(276729, 5)

['1844262553', '0615116426', '8826703132', '0618002227', '3423071516']

In [ ]:
len(["Blasphemous", "Blasphemous 2"])

2